In [1]:
import seaborn as sns
import pandas as pd
import pylab as plt
import numpy as np
import time
import os
import glob
import json
import tensorflow as tf
import sys
from IPython import display
from riotwatcher import RiotWatcher
import requests
from scipy.stats import zscore

api_file = os.path.join('..','..','..','apikeys','LOL_api_key.txt')
with open(api_file, 'r') as fin:
    watcher = RiotWatcher(fin.readline())

## Load hero data
and make cleaned_df with only desired information in it, formatted properly

In [2]:
champ_file=os.path.join('lol_data','version9.10.1_champion.json')
with open(champ_file,'r') as fin:
    champs_data=json.load(fin)
champ_vocab=[]
champ_names=[]
for name,champ in champs_data['data'].items():
    champ_vocab.append(champ['key'])
    champ_names.append(name)
champ_vocab = [int(i) for i in champ_vocab]
print(len(champ_vocab))

144


In [3]:
champ_df=pd.DataFrame(champs_data['data']).T
champ_df['key']=champ_df['key'].astype(int)
champ_df = champ_df.set_index('key')
#use pandas to expand dict of info and store in same df
champ_df = champ_df.join(champ_df['info'].apply(pd.Series))
champ_df = champ_df.join(champ_df['stats'].apply(pd.Series))
champ_df.head(1)

,blurb,id,image,info,name,partype,stats,tags,title,version,...,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
key,,,,,,,,,,,,,,,,,,,,,
266,Once honored defenders of Shurima against the ...,Aatrox,"{'full': 'Aatrox.png', 'sprite': 'champion0.pn...","{'attack': 8, 'defense': 4, 'magic': 3, 'diffi...",Aatrox,Blood Well,"{'hp': 580, 'hpperlevel': 90, 'mp': 0, 'mpperl...","[Fighter, Tank]",the Darkin Blade,9.10.1,...,3.0,0.75,0.0,0.0,0.0,0.0,60.0,5.0,2.5,0.651


In [4]:
champ_df.describe()

,attack,defense,magic,difficulty,hp,hpperlevel,mp,mpperlevel,movespeed,armor,...,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
count,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,...,144.000000,144.000000,144.000000,144.000000,144.0,144.0,144.000000,144.000000,144.000000,144.000000
mean,5.493056,4.750000,5.437500,5.715278,553.666389,88.355903,308.258750,32.399306,336.409722,30.032000,...,6.592014,0.660069,8.384472,0.557118,0.0,0.0,59.338524,3.253865,2.459979,0.648097
std,2.763388,2.077182,2.729581,2.057438,38.477250,6.697679,118.344494,16.667415,7.692893,6.732314,...,1.720340,0.162324,8.413705,0.282157,0.0,0.0,6.027883,0.601399,0.815094,0.036148
min,1.000000,1.000000,1.000000,1.000000,340.000000,65.000000,0.000000,0.000000,315.000000,17.040000,...,2.500000,0.300000,0.000000,0.000000,0.0,0.0,40.368000,1.500000,0.000000,0.475000
25%,3.000000,3.000000,3.000000,4.000000,531.500000,85.000000,276.380000,25.000000,330.000000,24.609000,...,5.500000,0.550000,6.675000,0.437500,0.0,0.0,55.000000,3.000000,2.000000,0.625000
50%,6.000000,5.000000,6.000000,6.000000,560.000000,88.000000,322.200000,37.000000,335.000000,30.000000,...,7.000000,0.600000,7.516000,0.625000,0.0,0.0,60.000000,3.170800,2.500000,0.644000
75%,8.000000,6.000000,8.000000,7.000000,582.060000,92.000000,377.250000,42.250000,340.000000,35.000000,...,8.000000,0.750000,8.010500,0.800000,0.0,0.0,64.000000,3.500000,3.000000,0.665000
max,10.000000,10.000000,10.000000,10.000000,625.640000,112.000000,500.000000,67.000000,355.000000,47.000000,...,10.000000,1.750000,50.000000,1.000000,0.0,0.0,69.180000,5.000000,6.000000,0.800000


In [5]:
info_columns = list(champ_df['info'].head(1))[0].keys()
stat_dict = list(champ_df['stats'].head(1))[0]
print(stat_dict)
bad_stats_keys=['mp','mpperlevel','mpregen','mpregenperlevel','crit','critperlevel']
for key in bad_stats_keys:
    print(stat_dict.pop(key))
stat_columns = list(stat_dict.keys())
print(stat_columns)

{'hp': 580, 'hpperlevel': 90, 'mp': 0, 'mpperlevel': 0, 'movespeed': 345, 'armor': 38, 'armorperlevel': 3.25, 'spellblock': 32.1, 'spellblockperlevel': 1.25, 'attackrange': 175, 'hpregen': 3, 'hpregenperlevel': 0.75, 'mpregen': 0, 'mpregenperlevel': 0, 'crit': 0, 'critperlevel': 0, 'attackdamage': 60, 'attackdamageperlevel': 5, 'attackspeedperlevel': 2.5, 'attackspeed': 0.651}
0
0
0
0
0
0
['hp', 'hpperlevel', 'movespeed', 'armor', 'armorperlevel', 'spellblock', 'spellblockperlevel', 'attackrange', 'hpregen', 'hpregenperlevel', 'attackdamage', 'attackdamageperlevel', 'attackspeedperlevel', 'attackspeed']


In [6]:
#convert tags to one hot using get_dummies and sum 
#    (some champs have two tags, some have one)
champ_tag_df=pd.DataFrame()
champ_tag_df[0] = pd.get_dummies(champ_df['tags'].apply(pd.Series)[0]).agg(np.array,axis=1)
champ_tag_df[1] = pd.get_dummies(champ_df['tags'].apply(pd.Series)[1]).agg(np.array,axis=1)
champ_tag_df['one_hot_tags'] = champ_tag_df.agg('sum',axis=1)

In [7]:
#cleaned df (for now) has only info or one hot tags.
champ_cleaned_df=pd.DataFrame()
#aggregate info to make into single column with list elements
champ_cleaned_df['info']=champ_df.filter(items=info_columns).apply(zscore).agg(np.array,axis=1)
champ_cleaned_df['stats']=champ_df.filter(items=stat_columns).apply(zscore).agg(np.array,axis=1)
champ_cleaned_df['one_hot_tags']=champ_tag_df['one_hot_tags']
champ_cleaned_df.sort_index(inplace=True)
champ_cleaned_df['index']=list(range(len(champ_cleaned_df.index)))
print(champ_cleaned_df.head(10))
champ_cleaned_df.describe()

                                                  info  \
key                                                      
1    [-1.268460070535441, -0.8454281663398984, 1.67...   
2    [1.2735036493447272, 0.12077545233427119, -0.8...   
3    [-1.6315977448040366, 2.536284499019695, 0.206...   
4    [0.18409062653894084, -1.328529975676983, 0.20...   
5    [0.9103659750761318, 0.6038772616713559, -0.89...   
6    [0.9103659750761318, 0.12077545233427119, -0.8...   
7    [-1.6315977448040366, -0.36232635700281357, 1....   
8    [-1.268460070535441, 0.6038772616713559, 0.942...   
9    [-1.268460070535441, -0.8454281663398984, 1.30...   
10   [0.18409062653894084, 0.6038772616713559, 0.57...   

                                                 stats        one_hot_tags  \
key                                                                          
1    [-0.7737023084848982, -0.05332370318574956, -0...  [0, 0, 1, 0, 0, 0]   
2    [1.1364040170833432, 0.6958092976677209, 1.772...  [0, 1, 0, 0, 

,index
count,144.000000
mean,71.500000
std,41.713307
min,0.000000
25%,35.750000
50%,71.500000
75%,107.250000
max,143.000000


# Load data and put it into a TFRecords file
Do this to nicely handle the features that is a numpy array (blue_champs and red_champs)

In [8]:
#get all files in challenger_tier data folder, then sort by date
files = glob.glob(os.path.join('LOL_data','*','V9.10','*.csv'))
#files.sort(key=os.path.getmtime)
print(files)

['LOL_data/challenger_tier/V9.10/saved2019-05-28.csv', 'LOL_data/masters_tier/V9.10/saved2019-05-17.csv']


In [9]:
lol_df = None
for filename in files[-2:]:
    print(filename)
    tier = filename[9:-31]
    if tier=='challenger':
        tier_num=0
    elif tier=='masters':
        tier_num=1
    else:
        raise KeyError
    
    if type(lol_df)==None:
        lol_df = pd.read_csv(filename, usecols=range(1,24))
        lol_df['skill_level'] = tier_num
    else:
        new_df = pd.read_csv(filename,usecols=range(1,24))
        new_df['skill_level'] = tier_num
        lol_df = pd.concat([lol_df, new_df],ignore_index=True)
lol_df.describe()

LOL_data/challenger_tier/V9.10/saved2019-05-28.csv
LOL_data/masters_tier/V9.10/saved2019-05-17.csv


,blue_win,blue_gold,red_gold,blue_tower_kills,red_tower_kills,blue_inhibitor_kills,red_inhibitor_kills,blue_dragon_kills,red_dragon_kills,blue_baron_kills,...,red_champs0,blue_champs1,red_champs1,blue_champs2,red_champs2,blue_champs3,red_champs3,blue_champs4,red_champs4,skill_level
count,441677.000000,441677.000000,441677.000000,441677.000000,441677.000000,441677.000000,441677.000000,441677.000000,441677.000000,441677.000000,...,441677.000000,441677.000000,441677.000000,441677.000000,441677.000000,441677.000000,441677.000000,441677.000000,441677.000000,441677.000000
mean,0.493564,50667.496270,50894.768367,5.239075,5.249990,0.856535,0.857602,1.333948,1.499933,0.424299,...,127.025784,128.486014,128.615937,128.043627,127.839944,128.491395,128.608868,127.264297,126.917019,0.724167
std,0.499959,15476.016762,15707.437316,3.728810,3.767128,1.113665,1.109356,1.214319,1.268132,0.613052,...,130.100459,130.826929,131.269630,130.474368,130.717533,130.736454,131.399606,130.096694,129.770730,0.446933
min,0.000000,3313.000000,3345.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,40860.000000,40873.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,0.000000
50%,0.000000,51309.000000,51651.000000,5.000000,5.000000,0.000000,0.000000,1.000000,1.000000,0.000000,...,84.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,1.000000
75%,1.000000,60938.000000,61422.000000,9.000000,9.000000,1.000000,1.000000,2.000000,2.000000,1.000000,...,145.000000,154.000000,154.000000,154.000000,150.000000,154.000000,154.000000,150.000000,145.000000,1.000000
max,1.000000,136791.000000,133619.000000,15.000000,15.000000,13.000000,12.000000,7.000000,9.000000,5.000000,...,555.000000,555.000000,555.000000,555.000000,555.000000,555.000000,555.000000,555.000000,555.000000,1.000000


In [28]:
lol_df_small = lol_df.head(1000)

In [10]:
print(lol_df.columns)

Index(['blue_win', 'blue_gold', 'red_gold', 'blue_tower_kills',
       'red_tower_kills', 'blue_inhibitor_kills', 'red_inhibitor_kills',
       'blue_dragon_kills', 'red_dragon_kills', 'blue_baron_kills',
       'red_baron_kills', 'gameid', 'region', 'blue_champs0', 'red_champs0',
       'blue_champs1', 'red_champs1', 'blue_champs2', 'red_champs2',
       'blue_champs3', 'red_champs3', 'blue_champs4', 'red_champs4',
       'skill_level'],
      dtype='object')


In [11]:
#cut games with gold totals that are too low,
#likely games that end without either teams getting gold
#beyond passive gold -> 500 starting gold + 2.04 per second after 1.67 minutes per hero
#-> 5 * (500 g/hero +2.04g/s *60 s/min (10 mins - 1.67 mins)) = 7500 gold
#see LOL_DataExploration for details
lol_df = lol_df.query('blue_gold > 9000')

In [12]:
#get dummies splits region into sparse columns corresponding to one hots
#    for each unique value in the original column. 
#agg(np.array) aggregates those sparse columns into a single sparse vector per match.
lol_df['one_hot_region'] = pd.get_dummies(lol_df['region']).agg(np.array,axis=1)

In [13]:
#merge info and tag columns for all champs
#insert filler merge to allow suffixes to trigger
#set suffix to false false so it throws a keyerror if merge has been performed
lol_df['info']=np.nan
lol_df['one_hot_tags']=np.nan
lol_df['index']=np.nan
lol_df['stats']=np.nan
for i in range(5):
    lol_df=pd.merge_ordered(lol_df,champ_cleaned_df,
                 left_on='blue_champs'+str(i),
                 right_on='key',
                 how='inner',
                 suffixes=('','_blue_champs'+str(i)))
    lol_df=pd.merge_ordered(lol_df,champ_cleaned_df,
                 left_on='red_champs'+str(i),
                 right_on='key',
                 how='inner',
                 suffixes=('','_red_champs'+str(i)))
    
lol_df = lol_df.drop(columns=['index',
                       'info',
                       'one_hot_tags',
                       'stats'])
lol_df = lol_df.sample(frac=1).reset_index(drop=True)


In [14]:
print('stat length:',len(lol_df['stats_blue_champs4'].head(1)[0]))

stat length: 14


In [15]:
print('region length:',len(lol_df['one_hot_region'].to_numpy()[0]))

region length: 11


In [16]:
lol_df.describe()

,blue_win,blue_gold,red_gold,blue_tower_kills,red_tower_kills,blue_inhibitor_kills,red_inhibitor_kills,blue_dragon_kills,red_dragon_kills,blue_baron_kills,...,index_blue_champs0,index_red_champs0,index_blue_champs1,index_red_champs1,index_blue_champs2,index_red_champs2,index_blue_champs3,index_red_champs3,index_blue_champs4,index_red_champs4
count,434110.000000,434110.000000,434110.000000,434110.000000,434110.000000,434110.000000,434110.000000,434110.000000,434110.000000,434110.000000,...,434110.000000,434110.000000,434110.000000,434110.000000,434110.000000,434110.000000,434110.000000,434110.000000,434110.000000,434110.000000
mean,0.493601,51465.905324,51695.890825,5.330391,5.340725,0.871466,0.872403,1.357195,1.526056,0.431695,...,74.329681,74.251780,74.698804,74.664032,74.606107,74.443019,74.733991,74.611898,74.354594,74.275819
std,0.499960,14368.983296,14612.972775,3.695898,3.735501,1.117523,1.113189,1.211910,1.263463,0.615785,...,42.311460,42.342447,42.379957,42.451213,42.327981,42.427115,42.365611,42.442789,42.316382,42.281978
min,0.000000,9037.000000,6670.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,41471.000000,41513.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000
50%,0.000000,51636.000000,51973.000000,5.000000,5.000000,0.000000,0.000000,1.000000,1.000000,0.000000,...,75.000000,74.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,74.000000
75%,1.000000,61122.000000,61617.000000,9.000000,9.000000,2.000000,2.000000,2.000000,2.000000,1.000000,...,112.000000,111.000000,113.000000,113.000000,113.000000,112.000000,113.000000,113.000000,112.000000,111.000000
max,1.000000,136791.000000,133619.000000,15.000000,15.000000,13.000000,12.000000,7.000000,9.000000,5.000000,...,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000


## Define preprocess functions

In [17]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _int64_list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
def _float_list_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [18]:
#target_name to name tfrecords file and the target_df column to call
#for now don't keep all possible targets in target_df
#target_df selection is made with the preprocess_target function call
def convert_to_tfrecords(filename, examples, target_df,target_name):
    # open the TFRecords file
    writer = tf.python_io.TFRecordWriter(filename)
    
    blue_hero_array = np.array(examples['blue_champs'])
    red_hero_array = np.array(examples['red_champs'])
    target_array = np.array(target_df[target_name])
    
    for i in range(len(blue_hero_array[:])):
        # print how many games are saved every 10000 games
        if not i % 10000:
            print('Train data: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the hero_array
        blue_champs = blue_hero_array[:][i].tostring()
        red_champs = red_hero_array[:][i].tostring()
        target = target_array[i]

        # Create a feature
        if target_name=='blue_win':
            feature = {'blue_champs': _bytes_feature(tf.compat.as_bytes(blue_champs)),
                   'red_champs': _bytes_feature(tf.compat.as_bytes(red_champs)),
                   'targets': _int64_feature(target)}
        else:
            feature = {'blue_champs': _bytes_feature(tf.compat.as_bytes(blue_champs)),
                   'red_champs': _bytes_feature(tf.compat.as_bytes(red_champs)),
                   'targets': _float_feature(target)}
        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    
    writer.close()
    sys.stdout.flush()

In [19]:
#target_name to name tfrecords file and the target_df column to call
#for now don't keep all possible targets in target_df
#target_df selection is made with the preprocess_target function call
def convert_to_tfrecords_info_tags_stats_region_dota2_compat(filename, examples, target_df):
    # open the TFRecords file for writing
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename,options=options)
    
    blue_champ_list=[None]*5;blue_champ_info_list=[None]*5
    blue_champ_tags_list=[None]*5;blue_champ_stats_list=[None]*5
    red_champ_list=[None]*5;red_champ_info_list=[None]*5
    red_champ_tags_list=[None]*5;red_champ_stats_list=[None]*5
    
    #load columns into 2D lists
    for i in range(5):
        blue_champ_list[i]=np.array(examples['index_blue_champs'+str(i)])
        blue_champ_info_list[i]=np.array(examples['info_blue_champs'+str(i)])
        blue_champ_tags_list[i]=np.array(examples['one_hot_tags_blue_champs'+str(i)])
        blue_champ_stats_list[i]=np.array(examples['stats_blue_champs'+str(i)])
        red_champ_list[i]=np.array(examples['index_red_champs'+str(i)])
        red_champ_info_list[i]=np.array(examples['info_red_champs'+str(i)])
        red_champ_tags_list[i]=np.array(examples['one_hot_tags_red_champs'+str(i)])
        red_champ_stats_list[i]=np.array(examples['stats_red_champs'+str(i)])
    
    region_array = np.array(examples['one_hot_region'])
    skill_level_array = np.array(examples['skill_level'])
    target_array = np.array(target_df[target_name])

    for i in range(len(blue_champ_list[0])):
        # print how many games are saved every 10000 games
        if not i % 10000:
            print('Train data: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        feature={}
        # Load the hero_array for each champ
        #    and add to feature dict
        num_champs=5
        for j in range(num_champs):
            blue_champ = blue_champ_list[j][i]
            blue_champ_info = blue_champ_info_list[j][i]
            blue_champ_tags = blue_champ_tags_list[j][i]
            blue_champ_stats = blue_champ_stats_list[j][i]
            red_champ = red_champ_list[j][i]
            red_champ_info = red_champ_info_list[j][i]
            red_champ_tags = red_champ_tags_list[j][i]
            red_champ_stats = red_champ_stats_list[j][i]
            feature['radiant_hero'+str(j)] = _int64_feature(blue_champ)
            feature['radiant_hero_info'+str(j)] = _float_list_feature(blue_champ_info)
            feature['radiant_hero_tags'+str(j)] = _int64_list_feature(blue_champ_tags)
            feature['radiant_hero_stats'+str(j)] = _float_list_feature(blue_champ_stats)
            feature['dire_hero'+str(j)] = _int64_feature(red_champ)
            feature['dire_hero_info'+str(j)] = _float_list_feature(red_champ_info)
            feature['dire_hero_tags'+str(j)] = _int64_list_feature(red_champ_tags)
            feature['dire_hero_stats'+str(j)] = _float_list_feature(red_champ_stats)
            
        
        
        #convert skill_level to one_hot
        skill_level = skill_level_array[i]
        
        #get region,target elements
        region = region_array[i]
        target = target_array[i]
        
        # Add feature elements that aren't for each player
        feature['region'] = _int64_list_feature(region)
        feature['skill_level'] = _int64_feature(skill_level)
        feature['targets'] = _int64_feature(target)

        
        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    
    writer.close()
    sys.stdout.flush()

In [20]:
#target_name to name tfrecords file and the target_df column to call
#for now don't keep all possible targets in target_df
#target_df selection is made with the preprocess_target function call
def convert_to_tfrecords_dota2_compat(filename, examples, target_df,target_name):
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename,options=options)
    
    blue_hero_array = np.array(examples['blue_champs'])
    red_hero_array = np.array(examples['red_champs'])
    skill_level_array = np.array(examples['skill_level'])
    target_array = np.array(target_df[target_name])

    for i in range(len(blue_hero_array[:])):
        # print how many games are saved every 10000 games
        if not i % 10000:
            print('Train data: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the hero_array
        blue_champs = blue_hero_array[:][i]
        red_champs = red_hero_array[:][i]
        
        #convert from hero id to just hero index (from 0-len(vocab))
        blue_champs_index=hero_to_hero_indices(blue_champs)
        red_champs_index=hero_to_hero_indices(red_champs)
            
        skill_level = skill_level_array[i]
        #convert skill_level to one_hot
        skill_level = tf.keras.utils.to_categorical(skill_level,
                                                    num_classes=2,
                                                    dtype='int64')

        target = target_array[i]
        
        # Create a feature
        if target_name=='blue_win':
            feature = {'radiant_heroes': _int64_list_feature(blue_champs_index),
                   'dire_heroes': _int64_list_feature(red_champs_index),
                   'skill_level': _int64_list_feature(skill_level),
                   'targets': _int64_feature(target)}
        else:
            feature = {'blue_champs': _int64_list_feature(blue_champs_index),
                   'red_champs': _int64_list_feature(red_champs_index),
                   'skill_level': _int64_list_feature(skill_level),
                   'targets': _float_feature(target)}
        
        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    
    writer.close()
    sys.stdout.flush()

### Multi target tfrecord saving functions

In [21]:
#target_name to name tfrecords file and the target_df column to call
#for now don't keep all possible targets in target_df
#target_df selection is made with the preprocess_target function call
def convert_to_tfrecords_dual(filename, examples, target_df):
    # open the TFRecords file
    writer = tf.python_io.TFRecordWriter(filename)
    
    blue_hero_array = np.array(examples['blue_champs'])
    red_hero_array = np.array(examples['red_champs'])
    win_array = np.array(target_df['blue_win'])
    gold_diff_array = np.array(target_df['gold_diff'])
    
    for i in range(len(blue_hero_array[:])):
        # print how many games are saved every 10000 games
        if not i % 10000:
            print('Train data: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the hero_array
        blue_champs = blue_hero_array[:][i].tostring()
        red_champs = red_hero_array[:][i].tostring()
        blue_win = win_array[i]
        gold_diff = gold_diff_array[i]
        
        # Create a feature
        feature = {'blue_champs': _bytes_feature(tf.compat.as_bytes(blue_champs)),
                   'red_champs': _bytes_feature(tf.compat.as_bytes(red_champs)),
                   'blue_win': _int64_feature(blue_win),
                   'gold_diff': _float_feature(gold_diff)}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    
    writer.close()
    sys.stdout.flush()

In [22]:
#target_name to name tfrecords file and the target_df column to call
#for now don't keep all possible targets in target_df
#target_df selection is made with the preprocess_target function call
def convert_to_tfrecords_dual_dota2_compat(filename, examples, target_df):
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename,options=options)
    
    blue_hero_array = np.array(examples['blue_champs'])
    red_hero_array = np.array(examples['red_champs'])
    skill_level_array = np.array(examples['skill_level'])

    win_array = np.array(target_df['blue_win'])
    gold_diff_array = np.array(target_df['gold_diff'])
    
    for i in range(len(blue_hero_array[:])):
        # print how many games are saved every 10000 games
        if not i % 10000:
            print('Train data: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the hero_array
        blue_champs = blue_hero_array[:][i]
        red_champs = red_hero_array[:][i]
        blue_win = win_array[i]
        gold_diff = gold_diff_array[i]
        skill_level = skill_level_array[i]

        #convert skill_level to one_hot
        skill_level = tf.keras.utils.to_categorical(skill_level,
                                                    num_classes=2,
                                                    dtype='int64')        
        blue_champs_index=[]
        red_champs_index=[]
        pass_outer=False
        counter=0
        for red_champ,blue_champ in zip(red_champs,blue_champs):
            #add heroes to the heroes by index list for this game
            try:
                red_champs_index.append(champ_vocab.index(red_champ))
                blue_champs_index.append(champ_vocab.index(blue_champ))
            #if the hero isnt in the vocab then break and pass so 
            #  the example isnt used for training
            except ValueError:
                counter+=1
                pass_outer = True
                break
        if pass_outer:
            pass_outer=False
            continue

        # Create a feature
        feature = {'radiant_heroes': _int64_list_feature(blue_champs_index),
                   'dire_heroes': _int64_list_feature(red_champs_index),
                   'skill_level': _int64_list_feature(skill_level),
                   'radiant_win': _int64_feature(blue_win),
                   'gold_diff': _float_feature(gold_diff)}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    
    writer.close()
    sys.stdout.flush()

In [23]:
#target_name to name tfrecords file and the target_df column to call
#for now don't keep all possible targets in target_df
#target_df selection is made with the preprocess_target function call
def convert_to_tfrecords_triple_dota2_compat(filename, examples, target_df):
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename,options=options)
    
    blue_hero_array = np.array(examples['blue_champs'])
    red_hero_array = np.array(examples['red_champs'])
    skill_array = np.array(examples['skill_level'])
    win_array = np.array(target_df['blue_win'])
    gold_diff_array = np.array(target_df['gold_diff'])
    total_gold_array = np.array(target_df['total_gold'])
    
    for i in range(len(blue_hero_array[:])):
        # print how many games are saved every 10000 games
        if not i % 10000:
            print('Train data: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the hero_array
        blue_champs = blue_hero_array[:][i]
        red_champs = red_hero_array[:][i]
        blue_win = win_array[i]
        gold_diff = gold_diff_array[i]
        total_gold = total_gold_array[i]
        
        skill_level = skill_array[i]
        #convert skill_level to one_hot
        
        skill_level = tf.keras.utils.to_categorical(skill_level,
                                                    num_classes=2,
                                                    dtype='int64')        
        counter=0
        blue_champs_index=[]
        red_champs_index=[]
        pass_outer=False
        for red_champ,blue_champ in zip(red_champs,blue_champs):
            #add heroes to the heroes by index list for this game
            try:
                red_champs_index.append(champ_vocab.index(red_champ))
                blue_champs_index.append(champ_vocab.index(blue_champ))
            #if the hero isnt in the vocab then break and pass so 
            #  the example isnt used for training
            except ValueError:
                counter+=1
                pass_outer = True
                break
        if pass_outer:
            pass_outer=False
            continue

        # Create a feature
        feature = {'radiant_heroes': _int64_list_feature(blue_champs_index),
                   'dire_heroes': _int64_list_feature(red_champs_index),
                   'skill_level': _int64_list_feature(skill_level),
                   'radiant_win': _int64_feature(blue_win),
                   'gold_diff': _float_feature(gold_diff),
                   'total_gold': _float_feature(total_gold)}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    print('Out of vocab:',counter)
    writer.close()
    sys.stdout.flush()

In [24]:
#target_name to name tfrecords file and the target_df column to call
#for now don't keep all possible targets in target_df
#target_df selection is made with the preprocess_target function call
def convert_to_tfrecords_penta_dota2_compat(filename, examples, target_df):
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename,options=options)
    
    blue_hero_array = np.array(examples['blue_champs'])
    red_hero_array = np.array(examples['red_champs'])
    skill_array = np.array(examples['skill_level'])
    win_array = np.array(target_df['blue_win'])
    gold_diff_array = np.array(target_df['gold_diff'])
    total_gold_array = np.array(target_df['total_gold'])
    towkill_diff_array = np.array(target_df['towkill_diff'])
    towkill_total_array = np.array(target_df['towkill_total'])
    
    for i in range(len(blue_hero_array[:])):
        # print how many games are saved every 10000 games
        if not i % 10000:
            print('Train data: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the hero_array
        blue_champs = blue_hero_array[:][i]
        red_champs = red_hero_array[:][i]
        blue_win = win_array[i]
        gold_diff = gold_diff_array[i]
        total_gold = total_gold_array[i]
        towkill_diff = towkill_diff_array[i]
        towkill_total = towkill_total_array[i]
        
        skill_level = skill_array[i]
        #convert skill_level to one_hot
        skill_level = tf.keras.utils.to_categorical(skill_level,
                                                    num_classes=2,
                                                    dtype='int64')        
        blue_champs_index=[]
        red_champs_index=[]
        pass_outer=False
        counter=0
        for j in range(5):
            #add heroes to the heroes by index list for this game
            try:
                red_champs_index.append(champ_vocab.index(red_champs[j]))
                blue_champs_index.append(champ_vocab.index(blue_champs[j]))
            #if the hero isnt in the vocab then break and pass so 
            #  the example isnt used for training
            except ValueError:
                counter+=1
                pass_outer = True
                break
        if pass_outer:
            pass_outer=False
            continue

        # Create a feature
        feature = {'radiant_heroes': _int64_list_feature(blue_champs_index),
                   'dire_heroes': _int64_list_feature(red_champs_index),
                   'skill_level': _int64_list_feature(skill_level),
                   'radiant_win': _int64_feature(blue_win),
                   'gold_diff': _float_feature(gold_diff),
                   'total_gold': _float_feature(total_gold),
                   'tower_damage_diff': _float_feature(towkill_diff),
                   'total_tower_damage': _float_feature(towkill_total)}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    
    writer.close()
    sys.stdout.flush()

### Preprocess functions

In [25]:
#def z_score(column):
#   return (column-column.mean())/column.std()
    
def preprocess_features_id_skill(lol_df):
    """Take lol_df and create a dataframe containing only the features for our model
            Args: 
                lol_df: Dataframe containing lol training and test data
            returns: 
                processed_df: pandas DataFrame containing only feature columns
    """
    
    #Use arrays of champs for each team as features rather than each hero
    #individually to help model fit?
    #This is an attempt to make the model more similar to the movie review
    #text analysis example in the Google ML Crash Course
    processed_df = pd.DataFrame()
    
    processed_df['blue_champs'] = list(np.array(lol_df.loc[:,['blue_champs0',
                                                'blue_champs1','blue_champs2',
                                                'blue_champs3','blue_champs4']]).astype(int))
    processed_df['red_champs'] = list(np.array(lol_df.loc[:,['red_champs0',
                                    'red_champs1','red_champs2',
                                    'red_champs3','red_champs4']]).astype(int))
    processed_df['skill_level'] = list(np.array(lol_df['skill_level']).astype(int))
    
    return processed_df

def preprocess_features_extra_info(lol_df):
    """Take lol_df and create a dataframe containing only the features for our model
            Args: 
                lol_df: Dataframe containing lol training and test data
            returns: 
                processed_df: pandas DataFrame containing only feature columns
                            which in this case are skill level, region,
                            red and blue champs, and info, tags, stats for each 
                            champ as separate columns.
    """
        
    processed_df = pd.DataFrame()
    for i in range(5):
        processed_df['index_blue_champs'+str(i)]=lol_df['index_blue_champs'+str(i)]
        processed_df['info_blue_champs'+str(i)]=lol_df['info_blue_champs'+str(i)]
        processed_df['stats_blue_champs'+str(i)]=lol_df['stats_blue_champs'+str(i)]
        processed_df['one_hot_tags_blue_champs'+str(i)]=lol_df['one_hot_tags_blue_champs'+str(i)]
        processed_df['index_red_champs'+str(i)]=lol_df['index_red_champs'+str(i)]
        processed_df['info_red_champs'+str(i)]=lol_df['info_red_champs'+str(i)]
        processed_df['one_hot_tags_red_champs'+str(i)]=lol_df[['one_hot_tags_red_champs'+str(i)]]
        processed_df['stats_red_champs'+str(i)]=lol_df[['stats_red_champs'+str(i)]]
    
    processed_df['skill_level'] = list(np.array(lol_df['skill_level']).astype(int))
    processed_df['one_hot_region'] = list(np.array(lol_df['one_hot_region']))

    return processed_df

In [26]:
def preprocess_targets_win(lol_df):
    """Take lol_df and create a dataframe containing only the targets for our model. (blue_win here)
            Args: 
                lol_df: Dataframe containing lol training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column
    """
    target_df = pd.DataFrame()
    target_df['blue_win'] = lol_df['blue_win']
    
    return target_df

def preprocess_targets_gold(lol_df):
    """Take lol_df and create a dataframe containing only the targets for our model 
        (normalized gold difference in this case)
            Args: 
                lol_df: Dataframe containing lol training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column (gold_diff)
    """
    target_df = pd.DataFrame()
    target_df['gold_diff'] = zscore(lol_df['blue_gold']-lol_df['red_gold'])
    return target_df

def preprocess_targets_dual_gold_win(lol_df):
    """Take lol_df and create a dataframe containing only the targets for our model 
        (normalized gold difference in this case)
            Args: 
                lol_df: Dataframe containing lol training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column (gold_diff)
    """
    target_df = pd.DataFrame()
    target_df['blue_win'] = lol_df['blue_win']
    target_df['gold_diff'] = lol_df['blue_gold']-lol_df['red_gold']
    target_df['gold_diff'] = z_score(target_df['gold_diff'])
    
    return target_df

def preprocess_targets_triple_gold_win(lol_df):
    """Take lol_df and create a dataframe containing only the targets for our model 
        (normalized gold difference in this case)
            Args: 
                lol_df: Dataframe containing lol training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column (gold_diff)
    """
    target_df = pd.DataFrame()
    target_df['blue_win'] = lol_df['blue_win']
    target_df['gold_diff'] = lol_df['blue_gold']-lol_df['red_gold']
    target_df['gold_diff'] = z_score(target_df['gold_diff'])
    target_df['total_gold'] = lol_df['blue_gold']+lol_df['red_gold']
    target_df['total_gold'] = z_score(target_df['total_gold'])

    return target_df

def preprocess_targets_penta_gold_win_tow(lol_df):
    """Take lol_df and create a dataframe containing only the targets for our model 
        (normalized gold difference in this case)
            Args: 
                lol_df: Dataframe containing lol training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column (gold_diff)
    """
    target_df = pd.DataFrame()
    target_df['blue_win'] = lol_df['blue_win']
    target_df['gold_diff'] = lol_df['blue_gold']-lol_df['red_gold']
    target_df['gold_diff'] = z_score(target_df['gold_diff'])
    target_df['total_gold'] = lol_df['blue_gold']+lol_df['red_gold']
    target_df['total_gold'] = z_score(target_df['total_gold'])
    target_df['towkill_diff'] = lol_df['blue_tower_kills']-lol_df['red_tower_kills']
    target_df['towkill_diff'] = z_score(target_df['towkill_diff'].astype('float32'))
    target_df['towkill_total'] = lol_df['blue_tower_kills']-lol_df['red_tower_kills']
    target_df['towkill_total'] = z_score(target_df['towkill_total'].astype('float32'))

    return target_df

#this can be easily modified for baron, dragon, or inhib kills
def preprocess_targets_towerkills(lol_df):
    """Take lol_df and create a dataframe containing only the targets for our model 
        (normalized tower_kill difference in this case)
            Args: 
                lol_df: Dataframe containing lol training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column (tower_kills)
    """

    target_df = pd.DataFrame()
    target_df['towkill_diff'] = z_score(lol_df['blue_tower_kills']\
                                        -lol_df['red_tower_kills'])
    return target_df

### Visualize data for each set

In [39]:
plotting_df = lol_df

display.display(plotting_df.iloc[:train_len,:].describe())
display.display(plotting_df.iloc[train_len:train_len+test_len,:].describe())
display.display(plotting_df.iloc[train_len+test_len:,:].describe())

NameError: name 'train_len' is not defined

## Export tfrecords

### Export win tfrecord

In [115]:
tier = 'mix_tier'
training_frac = 0.7
train_len = int(len(lol_df) * training_frac)
test_len = int((len(lol_df) - train_len)/1.5)
validation_len = len(lol_df) - train_len - test_len

target_name='blue_win'
train_features = preprocess_features(lol_df.iloc[:train_len,:])
test_features = preprocess_features(lol_df.iloc[train_len:train_len+test_len,:])
validation_features = preprocess_features(lol_df.iloc[train_len+test_len:,:])

if target_name=='blue_win':
    train_targets = preprocess_targets_win(lol_df.iloc[:train_len,:])
    test_targets = preprocess_targets_win(lol_df.iloc[train_len:train_len+test_len,:])
    validation_targets = preprocess_targets_win(lol_df.iloc[train_len+test_len:,:])
else:
    train_targets = preprocess_targets_gold(lol_df.iloc[:train_len,:])
    test_targets = preprocess_targets_gold(lol_df.iloc[train_len:train_len+test_len,:])
    validation_targets = preprocess_targets_gold(lol_df.iloc[train_len+test_len:,:])



convert_to_tfrecords(os.path.join('LOL_data',tier,'lol_training_data', 
                     target_name+'_lol_training_data.tfrecords'),
                     train_features,
                     train_targets,
                     target_name=target_name)

convert_to_tfrecords(os.path.join('LOL_data',tier,'lol_test_data', 
                     target_name+'_lol_test_data.tfrecords'),
                     test_features,
                     test_targets,
                     target_name=target_name)

convert_to_tfrecords(os.path.join('LOL_data',tier,'lol_validation_data',
                     target_name+'_lol_validation_data.tfrecords'),
                     validation_features,
                     validation_targets,
                     target_name=target_name)


Train data: 0/309173
Train data: 10000/309173
Train data: 20000/309173
Train data: 30000/309173
Train data: 40000/309173
Train data: 50000/309173
Train data: 60000/309173
Train data: 70000/309173
Train data: 80000/309173
Train data: 90000/309173
Train data: 100000/309173
Train data: 110000/309173
Train data: 120000/309173
Train data: 130000/309173
Train data: 140000/309173
Train data: 150000/309173
Train data: 160000/309173
Train data: 170000/309173
Train data: 180000/309173
Train data: 190000/309173
Train data: 200000/309173
Train data: 210000/309173
Train data: 220000/309173
Train data: 230000/309173
Train data: 240000/309173
Train data: 250000/309173
Train data: 260000/309173
Train data: 270000/309173
Train data: 280000/309173
Train data: 290000/309173
Train data: 300000/309173
Train data: 0/88336
Train data: 10000/88336
Train data: 20000/88336
Train data: 30000/88336
Train data: 40000/88336
Train data: 50000/88336
Train data: 60000/88336
Train data: 70000/88336
Train data: 80000/88

### Export win extra info dota2 compat tfrecord

In [31]:
tier = 'mix_tier'
training_frac = 0.7
train_len = int(len(lol_df) * training_frac)
test_len = int((len(lol_df) - train_len)/1.5)
validation_len = len(lol_df) - train_len - test_len

target_name='blue_win'
train_features = preprocess_features_extra_info(lol_df.iloc[:train_len,:])
test_features = preprocess_features_extra_info(lol_df.iloc[train_len:train_len+test_len,:])
validation_features = preprocess_features_extra_info(lol_df.iloc[train_len+test_len:,:])

train_targets = preprocess_targets_win(lol_df.iloc[:train_len,:])
test_targets = preprocess_targets_win(lol_df.iloc[train_len:train_len+test_len,:])
validation_targets = preprocess_targets_win(lol_df.iloc[train_len+test_len:,:])


convert_to_tfrecords_info_tags_stats_region_dota2_compat(os.path.join('LOL_data',
                                                                tier,'lol_training_data', 
                     target_name+'_extra_info_dota2_compat_lol_training_data.tfrecords'),
                     train_features,
                     train_targets)

convert_to_tfrecords_info_tags_stats_region_dota2_compat(os.path.join('LOL_data',
                                                                tier,'lol_test_data', 
                     target_name+'_extra_info_dota2_compat_lol_test_data.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords_info_tags_stats_region_dota2_compat(os.path.join('LOL_data',
                                                                tier,'lol_validation_data',
                     target_name+'_extra_info_dota2_compat_lol_validation_data.tfrecords'),
                     validation_features,
                     validation_targets)


Train data: 0/303877
Train data: 10000/303877
Train data: 20000/303877
Train data: 30000/303877
Train data: 40000/303877
Train data: 50000/303877
Train data: 60000/303877
Train data: 70000/303877
Train data: 80000/303877
Train data: 90000/303877
Train data: 100000/303877
Train data: 110000/303877
Train data: 120000/303877
Train data: 130000/303877
Train data: 140000/303877
Train data: 150000/303877
Train data: 160000/303877
Train data: 170000/303877
Train data: 180000/303877
Train data: 190000/303877
Train data: 200000/303877
Train data: 210000/303877
Train data: 220000/303877
Train data: 230000/303877
Train data: 240000/303877
Train data: 250000/303877
Train data: 260000/303877
Train data: 270000/303877
Train data: 280000/303877
Train data: 290000/303877
Train data: 300000/303877
Train data: 0/86822
Train data: 10000/86822
Train data: 20000/86822
Train data: 30000/86822
Train data: 40000/86822
Train data: 50000/86822
Train data: 60000/86822
Train data: 70000/86822
Train data: 80000/86

In [30]:
tier = 'mix_tier'
training_frac = 0.7
train_len = int(len(lol_df_small) * training_frac)
test_len = int((len(lol_df_small) - train_len)/1.5)
validation_len = len(lol_df_small) - train_len - test_len

target_name='blue_win'
train_features = preprocess_features_extra_info(lol_df_small.iloc[:train_len,:])
test_features = preprocess_features_extra_info(lol_df_small.iloc[train_len:train_len+test_len,:])
validation_features = preprocess_features_extra_info(lol_df_small.iloc[train_len+test_len:,:])

train_targets = preprocess_targets_win(lol_df_small.iloc[:train_len,:])
test_targets = preprocess_targets_win(lol_df_small.iloc[train_len:train_len+test_len,:])
validation_targets = preprocess_targets_win(lol_df_small.iloc[train_len+test_len:,:])


convert_to_tfrecords_info_tags_stats_region_dota2_compat(os.path.join('LOL_data',
                                                                tier,'lol_training_data', 
                     target_name+'_extra_info_dota2_compat_lol_training_data_small.tfrecords'),
                     train_features,
                     train_targets)

convert_to_tfrecords_info_tags_stats_region_dota2_compat(os.path.join('LOL_data',
                                                                tier,'lol_test_data', 
                     target_name+'_extra_info_dota2_compat_lol_test_data_small.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords_info_tags_stats_region_dota2_compat(os.path.join('LOL_data',
                                                                tier,'lol_validation_data',
                     target_name+'_extra_info_dota2_compat_lol_validation_data_small.tfrecords'),
                     validation_features,
                     validation_targets)


Train data: 0/700
Train data: 0/200
Train data: 0/100


### Export gold tfrecord

In [13]:
tier = 'mix_tier'
training_frac = 0.7
train_len = int(len(lol_df) * training_frac)
test_len = int((len(lol_df) - train_len)/1.5)
validation_len = len(lol_df) - train_len - test_len

target_name='gold_diff'
train_features = preprocess_features(lol_df.iloc[:train_len,:])
test_features = preprocess_features(lol_df.iloc[train_len:train_len+test_len,:])
validation_features = preprocess_features(lol_df.iloc[train_len+test_len:,:])

if target_name=='blue_win':
    train_targets = preprocess_targets_win(lol_df.iloc[:train_len,:])
    test_targets = preprocess_targets_win(lol_df.iloc[train_len:train_len+test_len,:])
    validation_targets = preprocess_targets_win(lol_df.iloc[train_len+test_len:,:])
else:
    train_targets = preprocess_targets_gold(lol_df.iloc[:train_len,:])
    test_targets = preprocess_targets_gold(lol_df.iloc[train_len:train_len+test_len,:])
    validation_targets = preprocess_targets_gold(lol_df.iloc[train_len+test_len:,:])

convert_to_tfrecords(os.path.join('LOL_data',tier,'lol_training_data', 
                     target_name+'_lol_training_data.tfrecords'),
                     train_features,
                     train_targets,
                     target_name=target_name)

convert_to_tfrecords(os.path.join('LOL_data',tier,'lol_test_data', 
                     target_name+'_lol_test_data.tfrecords'),
                     test_features,
                     test_targets,
                     target_name=target_name)

convert_to_tfrecords(os.path.join('LOL_data',tier,'lol_validation_data',
                     target_name+'_lol_validation_data.tfrecords'),
                     validation_features,
                     validation_targets,
                     target_name=target_name)


Train data: 0/309173
Train data: 10000/309173
Train data: 20000/309173
Train data: 30000/309173
Train data: 40000/309173
Train data: 50000/309173
Train data: 60000/309173
Train data: 70000/309173
Train data: 80000/309173
Train data: 90000/309173
Train data: 100000/309173
Train data: 110000/309173
Train data: 120000/309173
Train data: 130000/309173
Train data: 140000/309173
Train data: 150000/309173
Train data: 160000/309173
Train data: 170000/309173
Train data: 180000/309173
Train data: 190000/309173
Train data: 200000/309173
Train data: 210000/309173
Train data: 220000/309173
Train data: 230000/309173
Train data: 240000/309173
Train data: 250000/309173
Train data: 260000/309173
Train data: 270000/309173
Train data: 280000/309173
Train data: 290000/309173
Train data: 300000/309173
Train data: 0/88336
Train data: 10000/88336
Train data: 20000/88336
Train data: 30000/88336
Train data: 40000/88336
Train data: 50000/88336
Train data: 60000/88336
Train data: 70000/88336
Train data: 80000/88

### Export dual gold_win tfrecord

In [104]:
tier = 'mix_tier'
training_frac = 0.7
train_len = int(len(lol_df) * training_frac)
validation_len = int((len(lol_df) - train_len)/1.5)
test_len = len(lol_df) - train_len - validation_len

target_name='dual_gold_win'
train_features = preprocess_features(lol_df.iloc[:train_len,:])
test_features = preprocess_features(lol_df.iloc[train_len:train_len+test_len,:])
validation_features = preprocess_features(lol_df.iloc[train_len+test_len:,:])


train_targets = preprocess_targets_dual_gold_win(lol_df.iloc[:train_len,:])
test_targets = preprocess_targets_dual_gold_win(lol_df.iloc[train_len:train_len+test_len,:])
validation_targets = preprocess_targets_dual_gold_win(lol_df.iloc[train_len+test_len:,:])

convert_to_tfrecords_dual(os.path.join('LOL_data',tier,'lol_training_data', 
                     target_name+'_lol_training_data.tfrecords'),
                     train_features,
                     train_targets)
convert_to_tfrecords_dual(os.path.join('LOL_data',tier,'lol_test_data', 
                     target_name+'_lol_test_data.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords_dual(os.path.join('LOL_data',tier,'lol_validation_data',
                     target_name+'_lol_validation_data.tfrecords'),
                     validation_features,
                     validation_targets)

NameError: name 'convert_to_tfrecords_dual' is not defined

### Export dual model with dota compatibility for easy keras model reusing

In [203]:
tier = 'mix_tier'
training_frac = 0.7
train_len = int(len(lol_df) * training_frac)
validation_len = int((len(lol_df) - train_len)/1.5)
test_len = len(lol_df) - train_len - validation_len

target_name='dual_gold_win_dota2_compat'
train_features = preprocess_features(lol_df.iloc[:train_len,:])
test_features = preprocess_features(lol_df.iloc[train_len:train_len+test_len,:])
validation_features = preprocess_features(lol_df.iloc[train_len+test_len:,:])

train_targets = preprocess_targets_dual_gold_win(lol_df.iloc[:train_len,:])
test_targets = preprocess_targets_dual_gold_win(lol_df.iloc[train_len:train_len+test_len,:])
validation_targets = preprocess_targets_dual_gold_win(lol_df.iloc[train_len+test_len:,:])

convert_to_tfrecords_dual_dota2_compat(os.path.join('LOL_data',tier,'lol_training_data', 
                     target_name+'_lol_training_data.tfrecords'),
                     train_features,
                     train_targets)
convert_to_tfrecords_dual_dota2_compat(os.path.join('LOL_data',tier,'lol_test_data', 
                     target_name+'_lol_test_data.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords_dual_dota2_compat(os.path.join('LOL_data',tier,'lol_validation_data',
                     target_name+'_lol_validation_data.tfrecords'),
                     validation_features,
                     validation_targets)

Train data: 0/309173
Train data: 10000/309173
Train data: 20000/309173
Train data: 30000/309173
Train data: 40000/309173
Train data: 50000/309173
Train data: 60000/309173
Train data: 70000/309173
Train data: 80000/309173
Train data: 90000/309173
Train data: 100000/309173
Train data: 110000/309173
Train data: 120000/309173
Train data: 130000/309173
Train data: 140000/309173
Train data: 150000/309173
Train data: 160000/309173
Train data: 170000/309173
Train data: 180000/309173
Train data: 190000/309173
Train data: 200000/309173
Train data: 210000/309173
Train data: 220000/309173
Train data: 230000/309173
Train data: 240000/309173
Train data: 250000/309173
Train data: 260000/309173
Train data: 270000/309173
Train data: 280000/309173
Train data: 290000/309173
Train data: 300000/309173
Train data: 0/44168
Train data: 10000/44168
Train data: 20000/44168
Train data: 30000/44168
Train data: 40000/44168
Train data: 0/88336
Train data: 10000/88336
Train data: 20000/88336
Train data: 30000/88336


### Export triple win gold diff and total tfrecord

In [225]:
tier = 'mix_tier'
training_frac = 0.7
train_len = int(len(lol_df) * training_frac)
validation_len = int((len(lol_df) - train_len)/1.5)
test_len = len(lol_df) - train_len - validation_len

target_name='triple_gold_win_dota2_compat'
train_features = preprocess_features(lol_df.iloc[:train_len,:])
test_features = preprocess_features(lol_df.iloc[train_len:train_len+test_len,:])
validation_features = preprocess_features(lol_df.iloc[train_len+test_len:,:])


train_targets = preprocess_targets_triple_gold_win(lol_df.iloc[:train_len,:])
test_targets = preprocess_targets_triple_gold_win(lol_df.iloc[train_len:train_len+test_len,:])
validation_targets = preprocess_targets_triple_gold_win(lol_df.iloc[train_len+test_len:,:])

convert_to_tfrecords_triple_dota2_compat(os.path.join('LOL_data',tier,'lol_training_data', 
                     target_name+'_lol_training_data.tfrecords'),
                     train_features,
                     train_targets)
convert_to_tfrecords_triple_dota2_compat(os.path.join('LOL_data',tier,'lol_test_data', 
                     target_name+'_lol_test_data.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords_triple_dota2_compat(os.path.join('LOL_data',tier,'lol_validation_data',
                     target_name+'_lol_validation_data.tfrecords'),
                     validation_features,
                     validation_targets)

Train data: 0/309173
Train data: 10000/309173
Train data: 20000/309173
Train data: 30000/309173
Train data: 40000/309173
Train data: 50000/309173
Train data: 60000/309173
Train data: 70000/309173
Train data: 80000/309173
Train data: 90000/309173
Train data: 100000/309173
Train data: 110000/309173
Train data: 120000/309173
Train data: 130000/309173
Train data: 140000/309173
Train data: 150000/309173
Train data: 160000/309173
Train data: 170000/309173
Train data: 180000/309173
Train data: 190000/309173
Train data: 200000/309173
Train data: 210000/309173
Train data: 220000/309173
Train data: 230000/309173
Train data: 240000/309173
Train data: 250000/309173
Train data: 260000/309173
Train data: 270000/309173
Train data: 280000/309173
Train data: 290000/309173
Train data: 300000/309173
Out of vocab: 0
Train data: 0/44168
Train data: 10000/44168
Train data: 20000/44168
Train data: 30000/44168
Train data: 40000/44168
Out of vocab: 0
Train data: 0/88336
Train data: 10000/88336
Train data: 2000

### Export penta target tfrecord dota compat gold, tower (diff and total) and win

In [209]:
tier = 'mix_tier'
training_frac = 0.7
train_len = int(len(lol_df) * training_frac)
validation_len = int((len(lol_df) - train_len)/1.5)
test_len = len(lol_df) - train_len - validation_len

target_name='penta_gold_win_dota2_compat'
train_features = preprocess_features(lol_df.iloc[:train_len,:])
test_features = preprocess_features(lol_df.iloc[train_len:train_len+test_len,:])
validation_features = preprocess_features(lol_df.iloc[train_len+test_len:,:])


train_targets = preprocess_targets_penta_gold_win_tow(lol_df.iloc[:train_len,:])
test_targets = preprocess_targets_penta_gold_win_tow(lol_df.iloc[train_len:train_len+test_len,:])
validation_targets = preprocess_targets_penta_gold_win_tow(lol_df.iloc[train_len+test_len:,:])

convert_to_tfrecords_penta_dota2_compat(os.path.join('LOL_data',tier,'lol_training_data', 
                     target_name+'_lol_training_data.tfrecords'),
                     train_features,
                     train_targets)

convert_to_tfrecords_penta_dota2_compat(os.path.join('LOL_data',tier,'lol_test_data', 
                     target_name+'_lol_test_data.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords_penta_dota2_compat(os.path.join('LOL_data',tier,'lol_validation_data',
                     target_name+'_lol_validation_data.tfrecords'),
                     validation_features,
                     validation_targets)

Train data: 0/309173
Train data: 10000/309173
Train data: 20000/309173
Train data: 30000/309173
Train data: 40000/309173
Train data: 50000/309173
Train data: 60000/309173
Train data: 70000/309173
Train data: 80000/309173
Train data: 90000/309173
Train data: 100000/309173
Train data: 110000/309173
Train data: 120000/309173
Train data: 130000/309173
Train data: 140000/309173
Train data: 150000/309173
Train data: 160000/309173
Train data: 170000/309173
Train data: 180000/309173
Train data: 190000/309173
Train data: 200000/309173
Train data: 210000/309173
Train data: 220000/309173
Train data: 230000/309173
Train data: 240000/309173
Train data: 250000/309173
Train data: 260000/309173
Train data: 270000/309173
Train data: 280000/309173
Train data: 290000/309173
Train data: 300000/309173
Train data: 0/44168
Train data: 10000/44168
Train data: 20000/44168
Train data: 30000/44168
Train data: 40000/44168
Train data: 0/88336
Train data: 10000/88336
Train data: 20000/88336
Train data: 30000/88336
